# FM with MLflow

In [4]:
import os
import time
import random
import pandas as pd 
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from pathlib import Path

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [5]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(seed=42)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim


class DeepFM(nn.Module):
    def __init__(self, input_dims, embedding_dim, mlp_dims, drop_rate=0.1):
        super(DeepFM, self).__init__()
        total_input_dim = int(sum(input_dims))

        self.bias = nn.Parameter(torch.zeros((1,)))
        self.fc = nn.Embedding(total_input_dim, 1)
        
        self.embedding = nn.Embedding(total_input_dim, embedding_dim) 
        self.embedding_dim = len(input_dims) * embedding_dim

        mlp_layers = []
        for i, dim in enumerate(mlp_dims):
            if i==0:
                mlp_layers.append(nn.Linear(self.embedding_dim, dim))
            else:
                mlp_layers.append(nn.Linear(mlp_dims[i-1], dim))
            mlp_layers.append(nn.ReLU(True))
            mlp_layers.append(nn.Dropout(drop_rate))
        mlp_layers.append(nn.Linear(mlp_dims[-1], 1))
        self.mlp_layers = nn.Sequential(*mlp_layers)

    def fm(self, x):
        embed_x = self.embedding(x)
        fm_y = self.bias + torch.sum(self.fc(x), dim=1)
        square_of_sum = torch.sum(embed_x, dim=1) ** 2         
        sum_of_square = torch.sum(embed_x ** 2, dim=1) 
        fm_y += 0.5 * torch.sum(square_of_sum - sum_of_square, dim=1, keepdim=True)
        return fm_y
    
    def mlp(self, x):
        embed_x = self.embedding(x) 
        inputs = embed_x.view(-1, self.embedding_dim)
        mlp_y = self.mlp_layers(inputs)
        return mlp_y

    def forward(self, x):
        embed_x = self.embedding(x)
        fm_y = self.fm(x).squeeze(1)
        mlp_y = self.mlp(x).squeeze(1)
        y = torch.sigmoid(fm_y + mlp_y)
        return y


In [7]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader, Dataset



def FMpreprecessing(data_dir):
    train_ratings = pd.read_csv(f"{data_dir}/train/train_ratings.csv", sep=',')

    train_ratings['rating'] = 1.0 # implicit feedback
    train_ratings.drop(['time'],axis=1,inplace=True)

    users = set(train_ratings.loc[:, 'user'])
    items = set(train_ratings.loc[:, 'item'])

    num_negative = 50
    user_group_dfs = list(train_ratings.groupby('user')['item'])
    first_row = True
    user_neg_dfs = pd.DataFrame()

    for u, u_items in tqdm(user_group_dfs):
        u_items = set(u_items)
        i_user_neg_item = np.random.choice(list(items - u_items), num_negative, replace=False)
        
        i_user_neg_df = pd.DataFrame({'user': [u]*num_negative, 'item': i_user_neg_item, 'rating': [0]*num_negative})
        if first_row == True:
            user_neg_dfs = i_user_neg_df
            first_row = False
        else:
            user_neg_dfs = pd.concat([user_neg_dfs, i_user_neg_df], axis = 0, sort=False)

    train_ratings = pd.concat([train_ratings, user_neg_dfs], axis = 0, sort=False)

    users = list(set(train_ratings.loc[:,'user']))
    users.sort()
    items =  list(set((train_ratings.loc[:, 'item'])))
    items.sort()

    if len(users)-1 != max(users):
        users_dict = {users[i]: i for i in range(len(users))}
        train_ratings['user']  = train_ratings['user'].map(lambda x : users_dict[x])
        users = list(set(train_ratings.loc[:,'user']))
        
    if len(items)-1 != max(items):
        items_dict = {items[i]: i for i in range(len(items))}
        train_ratings['item']  = train_ratings['item'].map(lambda x : items_dict[x])
        items =  list(set((train_ratings.loc[:, 'item'])))

    data = train_ratings.sort_values(by=['user'])
    data.reset_index(drop=True, inplace=True)

    n_data = len(data)
    n_user = len(users)
    n_item = len(items)

    user_col = torch.tensor(data.loc[:,'user'])
    item_col = torch.tensor(data.loc[:,'item'])

    offsets = [0, n_user, n_user+n_item]
    for col, offset in zip([user_col, item_col], offsets):
        col += offset

    X = torch.cat([user_col.unsqueeze(1), item_col.unsqueeze(1)], dim=1)
    y = torch.tensor(list(data.loc[:,'rating']))

    return X, y, n_data, n_user, n_item




class FMDataset(Dataset):
    def __init__(self, input_tensor, target_tensor):
        self.input_tensor = input_tensor.long()
        self.target_tensor = target_tensor.long()

    def __getitem__(self, index):
        return self.input_tensor[index], self.target_tensor[index]

    def __len__(self):
        return self.target_tensor.size(0)


def FMdataloader(data_dir):
    print("Make Dataloader")
    X, y, n_data, n_user, n_item = FMpreprecessing(data_dir)

    dataset = FMDataset(X, y)
    train_ratio = 0.8

    train_size = int(train_ratio * n_data)
    test_size = n_data - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    return train_loader, test_loader, (n_user, n_item)

In [10]:
import os
import mlflow


class MlflowManager:
    '''
    MLflow 관리를 위한 클래스
    
    MLflow UI 실행 방법:
    mlflow ui -h 0.0.0.0 -p 30627

    Attributes:
        user_name: MLflow 사용자 이름 (str)
        tracking_uri: MLflow 추적 URI (str)
        experiment_name: MLflow 실험 이름 (str)

    Methods:
        init: MLflow 초기 설정
        start_run: MLflow 실행 시작
        log_params: 파라미터 로깅
        log_metric: 메트릭 로깅
        log_artifact: 아티팩트 로깅
        log_model: 모델 로깅
        end_run: MLflow 실행 종료
        autolog: MLflow 자동 로깅 설정
        get_tracking_uri: 현재 설정된 tracking URI 반환
        get_artifact_uri: 현재 아티팩트 URI 반환
    '''
    def __init__(self, user_name="root", tracking_uri="http://10.28.224.212", port=30627, experiment_name="tmp"):
        """
        MlflowManager 클래스 초기화

        Args:
            user_name (str): MLflow 사용자 이름
            tracking_uri (str): MLflow 추적 서버 URI
            port (int): MLflow 추적 서버 포트
            experiment_name (str): MLflow 실험 이름
        """
        self.user_name = user_name
        os.environ["LOGNAME"] = self.user_name
        self.tracking_uri = f"{tracking_uri}:{port}"
        self.experiment_name = experiment_name
        self.init()

    def init(self):
        """MLflow 초기 설정"""
        mlflow.set_tracking_uri(self.tracking_uri)
        mlflow.set_experiment(self.experiment_name)

    def start_run(self, run_name=None):
        """
        MLflow 실행 시작

        Args:
            run_name (str, optional): 실행 이름

        Returns:
            mlflow.ActiveRun: 활성화된 MLflow 실행 객체
        """
        return mlflow.start_run(run_name=run_name)

    def log_params(self, params):
        """
        파라미터 로깅

        Args:
            params (dict): 로깅할 파라미터 딕셔너리
        """
        mlflow.log_params(params)

    def log_metric(self, key, value, step):
        """
        메트릭 로깅

        Args:
            key (str): 메트릭 이름
            value (float): 메트릭 값
            step (int): 현재 스텝
        """
        mlflow.log_metric(key, value, step)

    def log_artifact(self, local_path):
        """
        아티팩트 로깅

        Args:
            local_path (str): 로깅할 아티팩트의 로컬 경로
        """
        mlflow.log_artifact(local_path)

    def log_model(self, model, artifact_path, type='torch'):
        """
        모델 로깅

        Args:
            model: 로깅할 모델 객체
            artifact_path (str): 아티팩트 저장 경로
            type (str): 모델 타입 ('torch' 또는 'lgb')

        Raises:
            Exception: 지원되지 않는 모델 타입일 경우 발생
        """
        if type == 'torch':
            mlflow.pytorch.log_model(model, artifact_path)
        elif type == 'lgb':
            mlflow.lightgbm.log_model(model, artifact_path)
        else:
            raise Exception(f"Check type: {type}")
        return None

    def end_run(self):
        """MLflow 실행 종료"""
        mlflow.end_run()

    def get_tracking_uri(self):
        """
        현재 설정된 tracking URI 반환

        Returns:
            str: 현재 설정된 tracking URI
        """
        return mlflow.get_tracking_uri()

    def get_artifact_uri(self):
        """
        현재 아티팩트 URI 반환

        Returns:
            str: 현재 아티팩트 URI
        """
        return mlflow.get_artifact_uri()
    

## Train

In [12]:
ml = MlflowManager(user_name="lagom", experiment_name="tmp")

with ml.start_run(run_name=None):

    device = torch.device('cuda')
    print("device: ", device)

    data_dir = "/data/ephemeral/home/level2-recsys-movierecommendation-recsys-04-lv3/.data"

    train_loader, val_loader, (n_user, n_item) = FMdataloader(data_dir)

    input_dims = [n_user, n_item]
    embedding_dim = 10
    lr = 0.001
    num_epochs = 5

    # Save MLflow log params
    # ml.log_params({"learning_rate": lr})
    # ml.log_params({"embedding_dim": embedding_dim})
    # ml.log_params({"num_epochs": num_epochs})

    params = {
        'learning_rate': lr,
        'embedding_dim': embedding_dim,
        'num_epochs': num_epochs
    }
    ml.log_params(params)

    model = DeepFM(input_dims, embedding_dim, mlp_dims=[30, 20, 10]).to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    min_val_loss = float('inf')
    for epoch in tqdm(range(num_epochs)):
        start = time.time()
        
        # Training
        model.train()
        train_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()

            output = model(x)
            loss = criterion(output, y.float())

            loss.backward()
            optimizer.step()
            train_loss += loss
        train_loss = train_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                output = model(x)
                loss = criterion(output, y.float())
                val_loss += loss.item()
        val_loss = val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # Save MLflow log metric
        ml.log_metric("time_to_train", time.time() - start, step=epoch)
        ml.log_metric("train_loss", train_loss, step=epoch)
        ml.log_metric("val_loss", val_loss, step=epoch)

        # Save model
        if min_val_loss > val_loss:
            min_val_loss = val_loss
            Path("/data/ephemeral/home/level2-recsys-movierecommendation-recsys-04-lv3/.saved/deepFM").mkdir(parents=True, exist_ok=True)
            torch.save(model, "/data/ephemeral/home/level2-recsys-movierecommendation-recsys-04-lv3/.saved/deepFM/deepFM_base.pt")

            # Save MLflow log model
            ml.log_model(model, "DeepFM_model", type='torch')


device:  cuda
Make Dataloader


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5 | Train Loss: 0.5979 | Val Loss: 0.3807


2024/11/19 08:29:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
 20%|██        | 1/5 [01:27<05:49, 87.46s/it]

Epoch 2/5 | Train Loss: 0.3769 | Val Loss: 0.3431


...

Epoch 4/5 | Train Loss: 0.3336 | Val Loss: 0.3340


2024/11/19 08:34:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
 80%|████████  | 4/5 [05:55<01:29, 89.30s/it]

Epoch 5/5 | Train Loss: 0.3255 | Val Loss: 0.3304


2024/11/19 08:35:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
100%|██████████| 5/5 [07:25<00:00, 89.17s/it]

🏃 View run unequaled-dove-244 at: http://10.28.224.212:30627/#/experiments/352035656253646964/runs/60d71e24c60a48edab84d4ed6502ec90
🧪 View experiment at: http://10.28.224.212:30627/#/experiments/352035656253646964
